<a href="https://colab.research.google.com/github/Mrmazenn1/ML-intern-at-Elevvo-/blob/main/loan-approval-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imblearn
!pip install xgboost

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import kagglehub


In [3]:
path = kagglehub.dataset_download("architsharma01/loan-approval-prediction-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'loan-approval-prediction-dataset' dataset.
Path to dataset files: /kaggle/input/loan-approval-prediction-dataset


In [4]:
df = pd.read_csv(f"{path}/loan_approval_dataset.csv")
df.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [5]:
df.describe()

,loan_id,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value
count,4269.000000,4269.000000,4.269000e+03,4.269000e+03,4269.000000,4269.000000,4.269000e+03,4.269000e+03,4.269000e+03,4.269000e+03
mean,2135.000000,2.498712,5.059124e+06,1.513345e+07,10.900445,599.936051,7.472617e+06,4.973155e+06,1.512631e+07,4.976692e+06
std,1232.498479,1.695910,2.806840e+06,9.043363e+06,5.709187,172.430401,6.503637e+06,4.388966e+06,9.103754e+06,3.250185e+06
min,1.000000,0.000000,2.000000e+05,3.000000e+05,2.000000,300.000000,-1.000000e+05,0.000000e+00,3.000000e+05,0.000000e+00
25%,1068.000000,1.000000,2.700000e+06,7.700000e+06,6.000000,453.000000,2.200000e+06,1.300000e+06,7.500000e+06,2.300000e+06
50%,2135.000000,3.000000,5.100000e+06,1.450000e+07,10.000000,600.000000,5.600000e+06,3.700000e+06,1.460000e+07,4.600000e+06
75%,3202.000000,4.000000,7.500000e+06,2.150000e+07,16.000000,748.000000,1.130000e+07,7.600000e+06,2.170000e+07,7.100000e+06
max,4269.000000,5.000000,9.900000e+06,3.950000e+07,20.000000,900.000000,2.910000e+07,1.940000e+07,3.920000e+07,1.470000e+07


In [6]:
df.shape

(4269, 13)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   loan_id                    4269 non-null   int64 
 1    no_of_dependents          4269 non-null   int64 
 2    education                 4269 non-null   object
 3    self_employed             4269 non-null   object
 4    income_annum              4269 non-null   int64 
 5    loan_amount               4269 non-null   int64 
 6    loan_term                 4269 non-null   int64 
 7    cibil_score               4269 non-null   int64 
 8    residential_assets_value  4269 non-null   int64 
 9    commercial_assets_value   4269 non-null   int64 
 10   luxury_assets_value       4269 non-null   int64 
 11   bank_asset_value          4269 non-null   int64 
 12   loan_status               4269 non-null   object
dtypes: int64(10), object(3)
memory usage: 433.7+ KB


In [8]:
numeric_cols=[' no_of_dependents', ' income_annum', ' loan_amount', ' loan_term', ' cibil_score', ' residential_assets_value', ' commercial_assets_value', ' luxury_assets_value', ' bank_asset_value']
for col in numeric_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

In [9]:
categorical_cols=[' education', ' self_employed', ' loan_status']
for col in categorical_cols:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)

In [10]:
print("Missing values in each column:")
print(df.isnull().sum())

Missing values in each column:
loan_id                      0
 no_of_dependents            0
 education                   0
 self_employed               0
 income_annum                0
 loan_amount                 0
 loan_term                   0
 cibil_score                 0
 residential_assets_value    0
 commercial_assets_value     0
 luxury_assets_value         0
 bank_asset_value            0
 loan_status                 0
dtype: int64


In [11]:
categorical_cols_to_encode = [' education', ' self_employed', ' loan_status']

In [12]:
df_encoded=pd.get_dummies(df, columns=categorical_cols_to_encode, drop_first=True)
print(df_encoded.head())

   loan_id   no_of_dependents   income_annum   loan_amount   loan_term  \
0        1                  2        9600000      29900000          12   
1        2                  0        4100000      12200000           8   
2        3                  3        9100000      29700000          20   
3        4                  3        8200000      30700000           8   
4        5                  5        9800000      24200000          20   

    cibil_score   residential_assets_value   commercial_assets_value  \
0           778                    2400000                  17600000   
1           417                    2700000                   2200000   
2           506                    7100000                   4500000   
3           467                   18200000                   3300000   
4           382                   12400000                   8200000   

    luxury_assets_value   bank_asset_value   education_ Not Graduate  \
0              22700000            8000000        

In [13]:
print("dataframe after encoding")
print(df_encoded.columns)

dataframe after encoding
Index(['loan_id', ' no_of_dependents', ' income_annum', ' loan_amount',
       ' loan_term', ' cibil_score', ' residential_assets_value',
       ' commercial_assets_value', ' luxury_assets_value', ' bank_asset_value',
       ' education_ Not Graduate', ' self_employed_ Yes',
       ' loan_status_ Rejected'],
      dtype='object')


In [14]:
class_counts = df_encoded[' loan_status_ Rejected'].value_counts()
print("class distribution of loan_ status:")
print(class_counts)
print ("percentage distribution:")
print(df_encoded[' loan_status_ Rejected'].value_counts(normalize=True)*100)


class distribution of loan_ status:
 loan_status_ Rejected
False    2656
True     1613
Name: count, dtype: int64
percentage distribution:
 loan_status_ Rejected
False    62.215976
True     37.784024
Name: proportion, dtype: float64


In [15]:
x=df_encoded.drop(columns=[' loan_status_ Rejected'])
y=df_encoded[' loan_status_ Rejected']


In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [17]:
model= RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [18]:
y_pred = model.predict(x_test)


In [19]:
print("confusion matrix")
print(pd.crosstab(y_test, y_pred))

confusion matrix
col_0                   False  True 
 loan_status_ Rejected              
False                     528      8
True                       12    306


In [20]:
print("classification report")
print(classification_report(y_test, y_pred))

classification report
              precision    recall  f1-score   support

       False       0.98      0.99      0.98       536
        True       0.97      0.96      0.97       318

    accuracy                           0.98       854
   macro avg       0.98      0.97      0.97       854
weighted avg       0.98      0.98      0.98       854



In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model_weighted = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model_weighted.fit(x_train, y_train)
y_pred_weighted = model_weighted.predict(x_test)
print("classification report with weighted classes")
print(classification_report(y_test, y_pred_weighted))

classification report with weighted classes
              precision    recall  f1-score   support

       False       0.97      0.99      0.98       536
        True       0.99      0.96      0.97       318

    accuracy                           0.98       854
   macro avg       0.98      0.98      0.98       854
weighted avg       0.98      0.98      0.98       854



In [22]:
feature_importances = model.feature_importances_
feature_names = x.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print("Feature Importance:")
print(feature_importance_df.head(10))
top_features = feature_importance_df.head(10)['Feature'].tolist()
x_top = x[top_features]

Feature Importance:
                     Feature  Importance
5                cibil_score    0.805016
4                  loan_term    0.061437
3                loan_amount    0.027802
6   residential_assets_value    0.016513
0                    loan_id    0.016491
2               income_annum    0.015770
8        luxury_assets_value    0.015550
7    commercial_assets_value    0.015024
9           bank_asset_value    0.013569
1           no_of_dependents    0.007873


In [23]:
param_grid = {
    'n_estimators': [100,200,300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']
}
Grid_search = GridSearchCV(RandomForestClassifier(random_state=42),param_grid,cv=3,scoring='recall')

Grid_search.fit(x_train, y_train)
print("Best Parameters:", Grid_search.best_params_)
print("Best Score:", Grid_search.best_score_)
best_params = Grid_search.best_params_
y_pred_best = Grid_search.predict(x_test)
print("classification report with best parameters")
print(classification_report(y_test, y_pred_best))

Best Parameters: {'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Best Score: 0.9799149981381227
classification report with best parameters
              precision    recall  f1-score   support

       False       0.99      0.98      0.98       536
        True       0.96      0.98      0.97       318

    accuracy                           0.98       854
   macro avg       0.97      0.98      0.98       854
weighted avg       0.98      0.98      0.98       854



In [26]:

x=df_encoded.drop(columns=[' loan_status_ Rejected'])
y=df_encoded[' loan_status_ Rejected']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train_resampled, y_train_resampled)
y_pred = model.predict(x_test)
print("classification report with SMOTE")
print(classification_report(y_test, y_pred))

classification report with SMOTE
              precision    recall  f1-score   support

       False       0.98      0.98      0.98       536
        True       0.97      0.97      0.97       318

    accuracy                           0.98       854
   macro avg       0.97      0.97      0.97       854
weighted avg       0.98      0.98      0.98       854



In [28]:
print ("logistic regression model")
lr_model=LogisticRegression(random_state=42 , solver='liblinear',class_weight='balanced')
lr_model.fit(x_train, y_train)
lr_pred = lr_model.predict(x_test)
print("confusion matrix:")
print(confusion_matrix(y_test, lr_pred))
print("classification report with logistic regression")
print(classification_report(y_test, lr_pred))

logistic regression model
confusion matrix:
[[449  87]
 [128 190]]
classification report with logistic regression
              precision    recall  f1-score   support

       False       0.78      0.84      0.81       536
        True       0.69      0.60      0.64       318

    accuracy                           0.75       854
   macro avg       0.73      0.72      0.72       854
weighted avg       0.74      0.75      0.74       854



In [30]:
from sklearn.tree import DecisionTreeClassifier
print("\n--- Decision Tree Model ---")
dt_model = DecisionTreeClassifier(random_state=42, class_weight='balanced')
dt_model.fit(x_train, y_train)


--- Decision Tree Model ---


DecisionTreeClassifier(class_weight='balanced', random_state=42)

In [33]:
dt_pred = dt_model.predict(x_test)

In [34]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, dt_pred))
print("\nClassification Report:")
print(classification_report(y_test, dt_pred))

Confusion Matrix:
[[525  11]
 [ 10 308]]

Classification Report:
              precision    recall  f1-score   support

       False       0.98      0.98      0.98       536
        True       0.97      0.97      0.97       318

    accuracy                           0.98       854
   macro avg       0.97      0.97      0.97       854
weighted avg       0.98      0.98      0.98       854

